In [85]:
from __future__ import print_function,division,absolute_import
import tables
import numpy as np
from keras.utils import to_categorical
from keras.models import model_from_json
from custom_layers import Conv1D_linearphase, DCT1D
from heartnet_v1 import reshape_folds
import pandas as pd
import os

In [35]:
log_name = 'fold1+compare 2018-05-05 17:04:36.995687'
checkpoint_name = "/media/taufiq/Data1/heart_sound/models/fold1+compare 2018-05-05 17:04:36.995687/weights.0007-0.8148.hdf5"
min_epoch = 3
min_metric = .7
foldname = 'fold1+compare'
fold_dir = '/media/taufiq/Data1/heart_sound/feature/segmented_noFIR/'
model_dir = '/media/taufiq/Data1/heart_sound/models/'
log_dir = '/media/taufiq/Data1/heart_sound/logs/'

## Load Model from checkpoint

In [ ]:
model_dir = checkpoint_name[:checkpoint_name.find('fold')]
log_name = checkpoint_name[checkpoint_name.find('fold'):checkpoint_name.find('weights')-1]
print(log_name)

In [ ]:
if os.path.isdir(model_dir+log_name):
    print("Model directory found")
    if os.path.isfile(os.path.join(model_dir+log_name,"model.json")):
        print("model.json found. Importing")
    else:
        raise ImportError("model.json not found")

In [ ]:
with open(os.path.join(model_dir+log_name,"model.json")) as json_file:
    loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json,{'Conv1D_linearphase':Conv1D_linearphase,'DCT1D':DCT1D})
model.load_weights(checkpoint_name)

## Load Data

In [152]:
############## Importing data ############
feat = tables.open_file(fold_dir + foldname + '.mat')
x_train = feat.root.trainX[:]
y_train = feat.root.trainY[0, :]
x_val = feat.root.valX[:]
y_val = feat.root.valY[0, :]
train_parts = feat.root.train_parts[0, :]
val_parts = feat.root.val_parts[0, :]

############## Relabeling ################

for i in range(0, y_train.shape[0]):
    if y_train[i] == -1:
        y_train[i] = 0  ## Label 0 for normal 1 for abnormal
for i in range(0, y_val.shape[0]):
    if y_val[i] == -1:
        y_val[i] = 0

############# Parse Database names ########

train_files = []
for each in feat.root.train_files[:][0]:
    train_files.append(chr(each))
print(len(train_files))
val_files = []
for each in feat.root.val_files[:][0]:
    val_files.append(chr(each))
print(len(val_files))

################### Reshaping ############

x_train, y_train, x_val, y_val = reshape_folds(x_train, x_val, y_train, y_val)
y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

3283
515
(93942, 2500, 1)
(93942, 1)
(15511, 2500, 1)
(15511, 1)


## Load Model from log name


Given a log_name it scans through the directories and finds the best three weights considering Sensitivity, Specificity and Macc.

In [93]:
if os.path.isdir(model_dir+log_name):
    print("Model directory found")
    if os.path.isfile(os.path.join(model_dir+log_name,"model.json")):
        print("model.json found. Importing")
    else:
        raise ImportError("model.json not found")

with open(os.path.join(model_dir+log_name,"model.json")) as json_file:
    loaded_model_json = json_file.read()
model = model_from_json(loaded_model_json,{'Conv1D_linearphase':Conv1D_linearphase,'DCT1D':DCT1D})
model.summary()

Model directory found
model.json found. Importing
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 2500, 1)      0                                            
__________________________________________________________________________________________________
conv1d_linearphase_1 (Conv1D_li (None, 2500, 1)      31          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_linearphase_2 (Conv1D_li (None, 2500, 1)      31          input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_linearphase_3 (Conv1D_li (None, 2500, 1)      31          input_1[0][0]                    
___________________________________________________________

In [94]:
training_csv = os.path.join(log_dir+log_name,"training.csv")
df = pd.read_csv(training_csv)

In [95]:
sens_idx = df['val_sensitivity'][df.epoch>min_epoch][df.val_specificity>min_metric].idxmax()
spec_idx = df['val_specificity'][df.epoch>min_epoch][df.val_sensitivity>min_metric].idxmax()
macc_idx = df['val_macc'][df.epoch>min_epoch].idxmax()


In [96]:
weights = dict()
weights['sens_weight'] = "weights.%.4d-%.4f.hdf5" % (df.epoch.iloc[sens_idx],df.val_acc.iloc[sens_idx])
weights['spec_weight'] = "weights.%.4d-%.4f.hdf5" % (df.epoch.iloc[spec_idx],df.val_acc.iloc[spec_idx])
weights['macc_weight'] = "weights.%.4d-%.4f.hdf5" % (df.epoch.iloc[macc_idx],df.val_acc.iloc[macc_idx])
print("Best Sensitivity model: \t{}".format(weights['sens_weight']))
print("Best Specificity model: \t{}".format(weights['spec_weight']))
print("Best Macc model: \t\t{}".format(weights['macc_weight']))

Best Sensitivity model: 	weights.0007-0.8059.hdf5
Best Specificity model: 	weights.0005-0.7794.hdf5
Best Macc model: 		weights.0007-0.8059.hdf5


## Load Training and Validation Filenames

In [68]:
import matlab.engine
eng = matlab.engine.start_matlab()
fold1_filenames = eng.load(os.path.join(fold_dir,'fold1_filenames.mat'))
compare_filenames = eng.load(os.path.join(fold_dir,'compare_filenames.mat'))
eng.quit()

train_filenames = fold1_filenames['train_files']
train_filenames.extend(compare_filenames['train_files'])
val_filenames = fold1_filenames['val_files']
val_filenames.extend(compare_filenames['val_files'])
print("Filenames Loaded : Train files {} and Validation Files {}".format(len(train_filenames),len(val_filenames)))

Filenames Loaded : Train files 3283 and Validation Files 515


In [81]:
dfTrain = pd.DataFrame(train_filenames,columns={"filenames"})
dfVal = pd.DataFrame(val_filenames,columns={"filenames"})
dfTrain['dataset'] = train_files
dfVal['dataset'] = val_files
dfVal.head()

,filenames,dataset
0,a0156.wav,a
1,a0148.wav,a
2,a0099.wav,a
3,b0265.wav,b
4,b0319.wav,b


## Get true labels per recording and append 

In [155]:
true = []
start_idx = 0
y_val_rec = np.transpose(np.argmax(y_val, axis=-1))
for s in val_parts:

    if not s:  ## for e00032 in validation0 there was no cardiac cycle
        continue
    # ~ print "part {} start {} stop {}".format(s,start_idx,start_idx+int(s)-1)

    temp_ = y_val_rec[start_idx:start_idx + int(s) - 1]

    if (sum(temp_ == 0) > sum(temp_ == 1)):
        true.append(0)
    else:
        true.append(1)

    start_idx = start_idx + int(s)
dfVal['true'] = true


true = []

start_idx = 0

y_val_rec = np.transpose(np.argmax(y_train, axis=-1))

for s in train_parts:

    if not s:  ## for e00032 in validation0 there was no cardiac cycle
        continue
    # ~ print "part {} start {} stop {}".format(s,start_idx,start_idx+int(s)-1)

    temp_ = y_val_rec[start_idx:start_idx + int(s) - 1]

    if (sum(temp_ == 0) > sum(temp_ == 1)):
        true.append(0)
    else:
        true.append(1)

    start_idx = start_idx + int(s)
dfTrain['true'] = true

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [160]:
train_parts[0,:]

array([36., 26., 25., ..., 34., 33., 32.])

In [120]:
y_pred = model.predict(x_val, verbose=1)
y_pred_hard = np.argmax(y_pred,axis=-1)
pred = []
start_idx = 0
y_val_rec = np.transpose(np.argmax(y_val, axis=-1))
for s in val_parts:

    if not s:  ## for e00032 in validation0 there was no cardiac cycle
        continue
    # ~ print "part {} start {} stop {}".format(s,start_idx,start_idx+int(s)-1)

    temp_ = y_val_rec[start_idx:start_idx + int(s) - 1]

    if (sum(temp_ == 0) > sum(temp_ == 1)):
        true.append(0)
    else:
        true.append(1)

    start_idx = start_idx + int(s)

15511/15511 [==============================] - 2s 115us/step


array([[0.6221047 , 0.3778953 ],
       [0.6336504 , 0.3663496 ],
       [0.64863044, 0.3513696 ],
       ...,
       [0.8927339 , 0.1072661 ],
       [0.94397974, 0.05602031],
       [0.9821339 , 0.01786613]], dtype=float32)

In [137]:
y_pred_soft = np.asarray([y_pred[i,j] for i,j in zip(range(len(y_pred)),np.nditer(y_pred_hard))])

In [149]:
.6+.3+.3

1.2